In [52]:
from langchain_community.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=QsYGlZkevEg",
    add_video_info=True,
    translation="en",
)
docs = loader.load()

In [53]:
print(docs)

[Document(metadata={'source': 'QsYGlZkevEg', 'title': 'Pedro Pascal Monologue - SNL', 'description': 'Unknown', 'view_count': 1854492, 'thumbnail_url': 'https://i.ytimg.com/vi/QsYGlZkevEg/hq720.jpg', 'publish_date': '2023-02-04 00:00:00', 'length': 224, 'author': 'Saturday Night Live'}, page_content='LADIES AND GENTLEMEN, PEDRO   PASCAL!  [ CHEERS AND APPLAUSE ]  >> THANK YOU, THANK YOU.  THANK YOU VERY MUCH.  I\'M SO EXCITED TO BE HERE.  THANK YOU.  I SPENT THE LAST YEAR SHOOTING A  SHOW CALLED "THE LAST OF US" ON   HBO.  FOR SOME HBO SHOES, YOU GET TO   SHOOT IN A FIVE STAR ITALIAN   RESORT SURROUNDED BY BEAUTIFUL   PEOPLE, BUT I SAID, NO, THAT\'S   TOO EASY.  I WANT TO SHOOT IN A FREEZING   CANADIAN FOREST WHILE BEING   CHASED AROUND BY A GUY WHOSE   HEAD LOOKS LIKE A GENITAL WART.  IT IS AN HONOR BEING A PART OF   THESE HUGE FRANCHISEs LIKE "GAME  OF THRONES" AND "STAR WARS," BUT  I\'M STILL GETTING USED TO PEOPLE  RECOGNIZING ME.  THE OTHER DAY, A GUY STOPPED ME   ON THE STREET AN

In [54]:
print((docs[0].page_content).lower())
print((docs[0].metadata))

ladies and gentlemen, pedro   pascal!  [ cheers and applause ]  >> thank you, thank you.  thank you very much.  i'm so excited to be here.  thank you.  i spent the last year shooting a  show called "the last of us" on   hbo.  for some hbo shoes, you get to   shoot in a five star italian   resort surrounded by beautiful   people, but i said, no, that's   too easy.  i want to shoot in a freezing   canadian forest while being   chased around by a guy whose   head looks like a genital wart.  it is an honor being a part of   these huge franchises like "game  of thrones" and "star wars," but  i'm still getting used to people  recognizing me.  the other day, a guy stopped me   on the street and says, my son   loves "the mandalorian" and the   next thing i know, i'm face   timing with a 6-year-old who has  no idea who i am because my   character wears a mask the   entire show.  the guy is like, do the mando   voice, but it's like a bedroom   voice.  without the mask, it just sounds  porny.  pe

In [55]:
print(f"author : {(docs[0].metadata['author'])}")
print(f"source: {docs[0].metadata['source']}")
print(f"title: {docs[0].metadata['title']}")
print(f"description: {docs[0].metadata['description']}")
print(f"view_count: {docs[0].metadata['view_count']}")
print(f"thumbnail_url: {docs[0].metadata['thumbnail_url']}")
print(f"publish_date: {docs[0].metadata['publish_date']}")
print(f"length: {docs[0].metadata['length']}")
print(f"author: {docs[0].metadata['author']}")

author : Saturday Night Live
source: QsYGlZkevEg
title: Pedro Pascal Monologue - SNL
description: Unknown
view_count: 1854492
thumbnail_url: https://i.ytimg.com/vi/QsYGlZkevEg/hq720.jpg
publish_date: 2023-02-04 00:00:00
length: 224
author: Saturday Night Live


In [56]:
from langchain_groq import ChatGroq
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
import os

llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768",api_key=os.environ["GROQ_KEY"])


In [71]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

@tool
def summarize(query: str) -> str:
    """Summarize the text"""
    user_query = query
    prompt_template = ChatPromptTemplate.from_messages(
        [
            ("system", 
            '''You are a smart assistant. 
            You can summarize that given text from a transcription of a youtube video, 
            title is {topic}.'''
            ),
            ("human", "transcription is {transcription}"),
        ]
    )

    # Create the combined chain using LangChain Expression Language (LCEL)
    chain = prompt_template | llm

    # Run the chain
    result = chain.invoke({"topic": docs[0].metadata["title"], "query": user_query, "transcription": docs[0].page_content})
    return result


@tool
def Quizme():
    """Create a quiz from the text"""
    prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a smart assistant.Given text is a transcription of a youtube video, 
                create a quiz with answers based on transcription. 5 questions will be ideal.
                video title is {topic}.""",
            ),
            ("human", "transcription is {transcription}"),
        ]
    )

    # Create the combined chain using LangChain Expression Language (LCEL)
    chain = prompt_template | llm

    # Run the chain
    result = chain.invoke(
        {
            "topic": docs[0].metadata["title"],
            "transcription": docs[0].page_content,
        }
    )
    return result

In [72]:
tools = [Quizme]
toolLlm = llm.bind_tools(tools)

In [73]:
result = toolLlm.invoke("Create a quiz about memory")
print(result)
print(result.tool_calls)
print(result.tool_calls[0]["name"])
print(result.tool_calls[0]["args"])

content='' additional_kwargs={'tool_calls': [{'id': 'call_q9fq', 'function': {'arguments': '{"topic":"memory"}', 'name': 'Quizme'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 1104, 'total_tokens': 1193, 'completion_time': 0.142759328, 'prompt_time': 0.060960082, 'queue_time': None, 'total_time': 0.20371941}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-0bfd7249-9ba0-4f63-99f4-87b1ab3d9190-0' tool_calls=[{'name': 'Quizme', 'args': {'topic': 'memory'}, 'id': 'call_q9fq', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1104, 'output_tokens': 89, 'total_tokens': 1193}
[{'name': 'Quizme', 'args': {'topic': 'memory'}, 'id': 'call_q9fq', 'type': 'tool_call'}]
Quizme
{'topic': 'memory'}


In [74]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

promt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
# Create the ReAct agent using the create_tool_calling_agent function
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=promt,
)

# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

# Test the agent with sample queries
response = agent_executor.invoke({"input": "I want to check my memory"})



> Entering new AgentExecutor chain...

Invoking: `Quizme` with `{'prompt': 'What is the memory I want to check?'}`


content="1. What show did Pedro Pascal spend the last year shooting?\nA. The Last of Us on HBO\n2. What did Pedro Pascal describe the creature chasing him in The Last of Us as?\nA. A guy whose head looks like a genital wart\n3. What did a father ask Pedro Pascal to do when his son recognized him from The Mandalorian?\nA. Do the Mando voice\n4. What did Pedro Pascal's cousins from Chile do that he had to correct?\nA. They gave out his phone number to everyone they meet and asked him questions about Baby Yoda\n5. What did Pedro Pascal's parents do that allowed him to be in the U.S. and standing on the SNL stage?\nA. They fled Chile and brought him and his sister to the U.S. nine months after he was born." response_metadata={'token_usage': {'completion_tokens': 192, 'prompt_tokens': 1151, 'total_tokens': 1343, 'completion_time': 0.309297708, 'prompt_time': 0.068624108, 'q

In [ ]:
print(response)

{'input': 'I want to check my memory', 'output': 'It seems you spent the last year shooting "The Last of Us" on HBO, choosing a freezing Canadian forest over a five-star Italian resort. You expressed your excitement for being on SNL and shared your experience with recognition from fans, including a FaceTime call with a 6-year-old who didn\'t recognize you without your mask from "The Mandalorian."\n\nYou also take pride in your Chilean roots, with 34 first cousins who are very proud of you. You recalled your early career with small parts in crime shows, showcasing your range as an actor. You expressed gratitude for your parents who brought you to the U.S. as a baby, allowing you to build a career in this wonderful country.\n\nTo conclude, you greeted your family in Chile and announced the exciting show ahead with Coldplay as the musical guest. It sounds like you had quite an adventure!'}


In [75]:
print(response["output"])

It seems like you want to check your memory on a quiz related to Pedro Pascal. Here's the content of the quiz:

1. What show did Pedro Pascal spend the last year shooting?
   a) The Last of Us on HBO
2. What did Pedro Pascal describe the creature chasing him in The Last of Us as?
   a) A guy whose head looks like a genital wart
3. What did a father ask Pedro Pascal to do when his son recognized him from The Mandalorian?
   a) Do the Mando voice
4. What did Pedro Pascal's cousins from Chile do that he had to correct?
   a) They gave out his phone number to everyone they meet and asked him questions about Baby Yoda
5. What did Pedro Pascal's parents do that allowed him to be in the U.S. and standing on the SNL stage?
   a) They fled Chile and brought him and his sister to the U.S. nine months after he was born.

Please let me know if you need any further assistance or if you're ready to provide the answers for me to check.
